In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import datetime
import time
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SPH6004/time_series.csv')
df.head()

,Time,stay_id,hadm_id,age,gender,Heart Rate,Respiratory Rate,SpO2/SaO2,pH,Potassium,Calcium,Glucose,Sodium,HCO3,White Blood Cells,Hemoglobin,Red Blood Cells,Platelet Count,Weight,Urea Nitrogen,Creatinine,Blood Pressure,1 hours urine output,6 hours urine output,Next 6 hours urine output,AKI,gcs,ventilation,vasoactive medications,sedative medications
0,2111-09-04 04:00:00,33225641,27845175,32,0,104.0,21.0,97.0,NaN,4.4,8.4,113.0,136.0,19.0,18.7,9.8,3.25,185.0,NaN,7.0,0.4,99.0,0.0,0.0,925.0,0,15.0,0,0,1
1,2111-09-04 05:00:00,33225641,27845175,32,0,104.0,25.0,95.0,7.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,550.0,0.0,995.0,0,NaN,0,0,1
2,2111-09-04 06:00:00,33225641,27845175,32,0,120.0,18.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.0,0.0,0.0,445.0,0,NaN,0,0,0
3,2111-09-04 07:00:00,33225641,27845175,32,0,106.0,18.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,275.0,0.0,520.0,0,NaN,0,0,1
4,2111-09-04 08:00:00,33225641,27845175,32,0,103.0,17.5,98.0,7.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,0.0,0.0,245.0,0,15.0,0,0,1


In [ ]:
pd.options.display.max_seq_items = None
df.columns

Index(['Time', 'stay_id', 'hadm_id', 'age', 'gender', 'Heart Rate',
       'Respiratory Rate', 'SpO2/SaO2', 'pH', 'Potassium', 'Calcium',
       'Glucose', 'Sodium', 'HCO3', 'White Blood Cells', 'Hemoglobin',
       'Red Blood Cells', 'Platelet Count', 'Weight', 'Urea Nitrogen',
       'Creatinine', 'Blood Pressure', '1 hours urine output',
       '6 hours urine output', 'Next 6 hours urine output', 'AKI', 'gcs',
       'ventilation', 'vasoactive medications', 'sedative medications'],
      dtype='object')

In [ ]:
date_time = pd.to_datetime(df.pop('Time'), format="%Y-%m-%d %H:%M:%S")
df = df.drop(['hadm_id'],axis=1)

In [ ]:
for i in df.columns:
  if any(np.isnan(df[i])):
    with tf.compat.v1.Session() as sess:
      df[i] = tf.where(tf.math.is_nan(df[i]), tf.zeros_like(df[i]), df[i]).eval()

'for i in df.columns:\n  if any(np.isnan(df[i])):\n    with tf.compat.v1.Session() as sess:\n      df[i] = tf.where(tf.math.is_nan(df[i]), tf.zeros_like(df[i]), df[i]).eval()'

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, data_raw, shift,
               label_columns=None):
    
    self.data_raw = data_raw
    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.feature_list = {name: i for i, name in
                           enumerate(df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.feature_list[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None, #have targets data here, every time point data
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def data_patient(self):
  return self.make_dataset(self.data_raw)

WindowGenerator.data_patient = data_patient

In [ ]:
patient_stay = df_norm['stay_id'].unique()
patient_num = len(patient_stay)
df_processed_inputs = []
df_processed_labels = []
for i in range(patient_num):
  df_patient = df_norm[df_norm['stay_id']==patient_stay[i]]
  df_patient = df_patient.drop(['stay_id'],axis=1)
  w = WindowGenerator(input_width=5, label_width=1, shift=1, data_raw =df_patient, label_columns=['AKI'])
  w.data_patient
  for x,y in w.data_patient:
    df_inputs = np.array(x)
    df_labels = np.array(y)
    df_processed_inputs.extend(df_inputs.copy())
    df_processed_labels.extend(df_labels.copy())
  print(i)

with open('/content/drive/MyDrive/SPH6004/df_processed_6.npy', 'wb') as f:
  np.save(f, np.array(df_processed_inputs))
  np.save(f, np.array(df_processed_labels))

  

In [ ]:
del df_norm
del df_patient
del df

In [ ]:
with open('/content/drive/MyDrive/SPH6004/df_processed_6.npy', 'rb') as f:
  df_processed_inputs = tf.convert_to_tensor(np.load(f))
  df_processed_labels = tf.convert_to_tensor(np.load(f))


n = len(df_processed_inputs)
train_df = [df_processed_inputs[0:int(n*0.7)], df_processed_labels[0:int(n*0.7)]]
val_df = [df_processed_inputs[int(n*0.7):int(n*0.9)], df_processed_labels[int(n*0.7):int(n*0.9)]]
test_df = [df_processed_inputs[int(n*0.9):], df_processed_labels[int(n*0.9):]]


In [ ]:
del df_processed_inputs
del df_processed_labels

tf.Tensor(
[[[0.]]

 [[0.]]

 [[0.]]

 ...

 [[0.]]

 [[1.]]

 [[0.]]], shape=(3771262, 1, 1), dtype=float32)


In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, train_inputs, train_labels, val_inputs, val_labels, patience=4):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                    min_delta=0.0001,
                                                    patience=patience,
                                                    mode='auto')

  model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                metrics= ['accuracy'] )

  history = model.fit(train_inputs, train_labels, epochs=MAX_EPOCHS,
                      validation_data=(val_inputs, val_labels),
                      callbacks=[early_stopping])
  return history


In [ ]:
val_performance = {}
performance = {}


In [ ]:
linear = tf.keras.Sequential([tf.keras.layers.Dense(units=1),
                              tf.keras.layers.Dropout(0.1)]
                             )
history = compile_and_fit(linear, train_inputs = train_df[0], train_labels = 
                          train_df[1], val_inputs = val_df[0], val_labels = val_df[1])

val_performance['Linear'] = linear.evaluate(val_df[0], val_df[1])
performance['Linear'] = linear.evaluate(test_df[0], test_df[1], verbose=0)

In [ ]:
print(val_performance, performance)

In [ ]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(units=1),
    tf.keras.layers.Dropout(0.1)
])

In [ ]:
history = compile_and_fit(lstm_model, train_inputs = train_df[0], train_labels = 
                          train_df[1], val_inputs = val_df[0], val_labels = val_df[1])


In [ ]:
val_performance['LSTM'] = lstm_model.evaluate(val_df[0],val_df[1])
performance['LSTM'] = lstm_model.evaluate(test_df[0],test_df[1], verbose=0)

In [ ]:
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=3,
                           kernel_size=3),
    tf.keras.layers.Dense(units=1),
    tf.keras.layers.Dropout(0.1)])

In [ ]:
history = compile_and_fit(conv_model, train_inputs = train_df[0], train_labels = 
                          train_df[1], val_inputs = val_df[0], val_labels = val_df[1])

Epoch 1/20
82497/82497 [==============================] - 143s 2ms/step - loss: 0.7798 - accuracy: 0.7682 - val_loss: 0.4758 - val_accuracy: 0.7736
Epoch 2/20
82497/82497 [==============================] - 138s 2ms/step - loss: 0.7546 - accuracy: 0.7753 - val_loss: 0.4707 - val_accuracy: 0.7737
Epoch 3/20
82497/82497 [==============================] - 142s 2ms/step - loss: 0.7562 - accuracy: 0.7748 - val_loss: 0.4687 - val_accuracy: 0.7738
Epoch 4/20
82497/82497 [==============================] - 140s 2ms/step - loss: 0.7555 - accuracy: 0.7748 - val_loss: 0.4693 - val_accuracy: 0.7739
Epoch 5/20
82497/82497 [==============================] - 140s 2ms/step - loss: 0.7525 - accuracy: 0.7752 - val_loss: 0.4692 - val_accuracy: 0.7742
Epoch 6/20
82497/82497 [==============================] - 141s 2ms/step - loss: 0.7541 - accuracy: 0.7752 - val_loss: 0.4698 - val_accuracy: 0.7737
Epoch 7/20
82497/82497 [==============================] - 139s 2ms/step - loss: 0.7554 - accuracy: 0.7748 - val_

In [ ]:
print(val_performance, performance)

In [ ]:
linear.save('/content/drive/MyDrive/SPH6004/linear6')
lstm_model.save('/content/drive/MyDrive/SPH6004/lstm6')
conv_model.save('/content/drive/MyDrive/SPH6004/conv6')

In [ ]:
linear_load = tf.keras.models.load_model('/content/drive/MyDrive/SPH6004/linear')
linear6_load = tf.keras.models.load_model('/content/drive/MyDrive/SPH6004/linear6')
lstm_load = tf.keras.models.load_model('/content/drive/MyDrive/SPH6004/lstm')
lstm6_load = tf.keras.models.load_model('/content/drive/MyDrive/SPH6004/lstm6')
conv_load = tf.keras.models.load_model('/content/drive/MyDrive/SPH6004/conv6')

In [ ]:
y = np.array(test_df[1])
y = np.array(test_df[1]).reshape(len(y)).astype(int)

y_pro_linear = np.array(linear_load(test_df[0])[:,-1,:]).reshape(len(y))
print(metrics.roc_curve(y, y_pro_linear), '\n linear AUC:',metrics.roc_auc_score(y, y_pro_linear))
y_pred_linear = np.array(np.where(linear_load(test_df[0])[:,-1,:]>0.5, 1, 0)).reshape(len(y))
print(metrics.confusion_matrix(y,y_pred_linear), '\n linear acc:', metrics.accuracy_score(y, y_pred_linear))

y_pro_linear6 = np.array(linear6_load(test_df[0])[:,-1,:]).reshape(len(y))
print(metrics.roc_curve(y, y_pro_linear6), '\n linear6 AUC:',metrics.roc_auc_score(y, y_pro_linear6))
y_pred_linear6 = np.array(np.where(linear6_load(test_df[0])[:,-1,:]>0.5, 1, 0)).reshape(len(y))
print(metrics.confusion_matrix(y,y_pred_linear6), '\n linear6 acc:', metrics.accuracy_score(y, y_pred_linear6))

y_pro_lstm = np.array(lstm_load(test_df[0])[:,-1,:]).reshape(len(y))
print(metrics.roc_curve(y, y_pro_lstm), '\n lstm AUC:',metrics.roc_auc_score(y, y_pro_lstm))
y_pred_lstm = np.array(np.where(lstm_load(test_df[0])[:,-1,:]>0.5, 1, 0)).reshape(len(y))
print(metrics.confusion_matrix(y,y_pred_lstm), '\n lstm acc:', metrics.accuracy_score(y, y_pred_lstm))

y_pro_lstm6 = np.array(lstm6_load(test_df[0])[:,-1,:]).reshape(len(y))
print(metrics.roc_curve(y, y_pro_lstm6), '\n lstm6 AUC:',metrics.roc_auc_score(y, y_pro_lstm6))
y_pred_lstm6 = np.array(np.where(lstm6_load(test_df[0])[:,-1,:]>0.5, 1, 0)).reshape(len(y))
print(metrics.confusion_matrix(y,y_pred_lstm6), '\n lstm6 acc:', metrics.accuracy_score(y, y_pred_lstm6))

y_pro_conv = np.array(conv_load(test_df[0])[:,-1,:]).reshape(len(y))
print(metrics.roc_curve(y, y_pro_conv), '\n conv AUC:',metrics.roc_auc_score(y, y_pro_conv))
y_pred_conv = np.array(np.where(conv_load(test_df[0])[:,-1,:]>0.5, 1, 0)).reshape(len(y))
print(metrics.confusion_matrix(y,y_pred_conv), '\n conv acc:', metrics.accuracy_score(y, y_pred_conv))

(array([0.00000000e+00, 3.20415258e-06, 1.60207629e-05, ...,
       9.99253432e-01, 9.99259841e-01, 1.00000000e+00]), array([0., 0., 0., ..., 1., 1., 1.]), array([ 1.5577159 ,  0.55771595,  0.52859694, ..., -0.03129405,
       -0.03133518, -0.11564982], dtype=float32)) 
 linear AUC: 0.6731805590176495
[[312087      8]
 [ 65029      3]] 
 linear acc: 0.8275461581907421
(array([0.00000000e+00, 3.20415258e-06, 1.92249155e-05, ...,
       9.99996796e-01, 9.99996796e-01, 1.00000000e+00]), array([0.        , 0.        , 0.        , ..., 0.99998462, 1.        ,
       1.        ]), array([ 1.5744383 ,  0.5744383 ,  0.53811204, ..., -0.16845816,
       -0.17085299, -0.22739601], dtype=float32)) 
 linear6 AUC: 0.5907359011211364
[[312075     20]
 [ 65029      3]] 
 linear6 acc: 0.8275143386710577
(array([0.00000000e+00, 3.20415258e-06, 3.20415258e-06, ...,
       9.97468719e-01, 9.97468719e-01, 1.00000000e+00]), array([0.00000000e+00, 0.00000000e+00, 1.23016361e-04, ...,
       9.99984623e-01, 